# RAG

Implement a base RAG module in DSPy. 
Given a question, retrieve the top-k documents in a list of HTML documents, then pass them as context to an LLM.

Refer to https://dspy.ai/tutorials/rag/. 


In [1]:
import dspy
import torch
from sentence_transformers import SentenceTransformer

# Load an extremely efficient local model for retrieval
_device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer("sentence-transformers/static-retrieval-mrl-en-v1", device=_device)

# Create an embedder using the model's encode method
embedder = dspy.Embedder(model.encode)

# Traverse a directory and read html files - extract text from the html files
import os
from bs4 import BeautifulSoup

def read_html_files(directory, sources_root="../PragmatiCQA-sources"):
    """
    Read .html files from a path. Accepts:
      - absolute path
      - relative path (from current working directory)
      - topic name or folder name (will be resolved under sources_root)
    Returns a list of document texts. If path not found, returns [].
    """
    texts = []

    # Normalize and resolve path
    if os.path.isabs(directory):
        path = directory
    else:
        # prefer the provided relative path if it exists
        if os.path.exists(directory):
            path = directory
        else:
            # treat `directory` as a topic name under the default sources root
            path = os.path.join(sources_root, directory)

    path = os.path.abspath(path)

    if not os.path.exists(path):
        # not throwing here so notebook flow is uninterrupted; caller can handle empty list
        print(f"Directory not found: {path}")
        return texts

    for filename in os.listdir(path):
        if filename.endswith(".html"):
            file_path = os.path.join(path, filename)
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    html = file.read()
                    soup = BeautifulSoup(html, 'html.parser')
                    texts.append(soup.get_text())
            except Exception as e:
                # skip problematic files but report minimal info
                print(f"Warning: could not read {file_path}: {e}")
    return texts

In [2]:
# Provide either an absolute path, a relative path, or a topic folder name.
# Examples:
#   directory = '/full/path/to/PragmatiCQA-sources/The Legend of Zelda'
#   directory = '../PragmatiCQA-sources/The Legend of Zelda'
#   directory = 'The Legend of Zelda'           # treated as topic name under sources_root

directory = "../PragmatiCQA-sources/The Legend of Zelda"
corpus = read_html_files(directory)
if not corpus:
    print(f"No documents loaded from: {os.path.abspath(directory)}")
else:
    print(f"Loaded {len(corpus)} documents. Will encode them below.")

Loaded 406 documents. Will encode them below.


In [3]:
# Parameters for the retriever
max_characters = 10000  # for truncating >99th percentile of documents
topk_docs_to_retrieve = 5  # number of documents to retrieve per search query

search = dspy.retrievers.Embeddings(embedder=embedder, corpus=corpus, k=topk_docs_to_retrieve)



In [4]:
# lm = dspy.LM('ollama_chat/devstral', api_base='http://localhost:11434', api_key='')
lm = dspy.LM('xai/grok-3-mini')
dspy.configure(lm=lm)

In [5]:
class RAG(dspy.Module):
    def __init__(self):
        self.respond = dspy.ChainOfThought('context, question -> response')

    def forward(self, question):
        context = search(question).passages
        return self.respond(context=context, question=question)
    
rag = RAG()

In [6]:
answer = rag(question="What is the main plot of The Legend of Zelda?")  # Example query

print(answer.response)  # Print the response from the RAG model

The main plot of The Legend of Zelda revolves around a young hero named Link who must save the kingdom of Hyrule from the evil Ganon, the Prince of Darkness. Ganon steals the Triforce of Power and seeks to conquer Hyrule, prompting Princess Zelda to break the Triforce of Wisdom into eight fragments and hide them across the land. Zelda sends her nursemaid, Impa, to find a brave warrior to stop Ganon. Link embarks on a quest to collect the fragments, explore dungeons, battle enemies, and ultimately confront Ganon to rescue Princess Zelda and restore peace to Hyrule.


In [7]:
q = 'What year did the Legend of Zelda come out?' 

print(rag(question=q).response)

The Legend of Zelda was first released in 1986.


Part 4.3 — Traditional QA baseline (start)

Plan:
- Use the existing retriever `search` to obtain retrieved passages for a question.
- Use Hugging Face's 'distilbert-base-cased-distilled-squad' extractive QA pipeline to answer the question given:
  1) Literal spans (from dataset),
  2) Pragmatic spans (from dataset),
  3) Retrieved context (from `search`).
- Evaluate these three configurations with dspy.evaluate.SemanticF1 on the first question of each conversation.

In [8]:
# 4.3 Part 1: Traditional QA Baseline Implementation
from transformers import pipeline
import json, os
from tqdm.notebook import tqdm
import pandas as pd
import dspy
from dspy.evaluate import SemanticF1

def setup_qa_pipeline(device=None):
    """Initialize a robust QA pipeline.
    Tries a list of reliable HF models, detects CUDA availability,
    avoids float16 on CPU, and falls back gracefully to a CPU pipeline.
    Accepts device as: 'cuda', 0, -1, or None.
    """
    import torch
    from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline as _pipeline

    # Reliable candidate models
    model_candidates = [
        'distilbert/distilbert-base-cased-distilled-squad',
        'deepset/roberta-base-squad2'
    ]

    last_err = None
    # Normalize device intent
    want_cuda = False
    if isinstance(device, str) and device.lower().startswith('cuda'):
        want_cuda = True
    elif isinstance(device, int) and device >= 0:
        want_cuda = True

    for model_name in model_candidates:
        try:
            tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
            # Only use GPU if available
            use_cuda = want_cuda and torch.cuda.is_available()
            if use_cuda:
                model = AutoModelForQuestionAnswering.from_pretrained(model_name).half().to('cuda')
                device_id = 0
            else:
                model = AutoModelForQuestionAnswering.from_pretrained(model_name)
                device_id = -1
            model.eval()
            return _pipeline('question-answering', model=model, tokenizer=tokenizer, device=device_id)
        except Exception as e:
            last_err = e
            # try next candidate
            continue

    # Final fallback: let transformers download and create a CPU pipeline from a known model id
    try:
        return _pipeline('question-answering', model=model_candidates[0], device=-1)
    except Exception as e:
        print('Error setting up QA pipeline: ', last_err)
        print('Final fallback failed: ', e)
        # Re-raise to make failure visible to the caller
        raise

def load_validation_data(filename="val.jsonl", dataset_dir="../PragmatiCQA/data"):
    """Load and filter validation data to first questions only."""
    try:
        with open(os.path.join(dataset_dir, filename), 'r') as f:
            return [json.loads(line) for line in f]
    except Exception as e:
        print(f"Error loading validation data: {e}")
        return []

def evaluate_qa_approaches(val_data, search, qa_pipeline, metric, max_samples=None):
    """
    Evaluate three QA approaches on first questions:
    1. Using literal spans
    2. Using pragmatic spans
    3. Using retrieved context
    """
    results = []
    samples = val_data[:max_samples] if max_samples else val_data
    
    for doc in tqdm(samples, desc="Evaluating QA approaches"):
        if not doc.get("qas"):
            continue
            
        # Get first question data
        qa = doc["qas"][0]
        question = qa["q"]
        gold = qa["a"]
        
        # Setup contexts
        literal_spans = [s["text"] for s in qa["a_meta"].get("literal_obj", [])]
        pragmatic_spans = [s["text"] for s in qa["a_meta"].get("pragmatic_obj", [])]
        literal_context = " ".join(literal_spans).strip()
        pragmatic_context = " ".join(pragmatic_spans).strip()
        
        try:
            topic = doc.get("topic", "")
            topic_corpus = read_html_files(topic)
            if not topic_corpus:
                print(f"No corpus found for topic '{topic}', skipping question: {question}")
                continue
            topic_search = dspy.retrievers.Embeddings(embedder=embedder, corpus=topic_corpus, k=topk_docs_to_retrieve)
            retrieved_passages = topic_search(question).passages
            retrieved_context = " ".join(retrieved_passages).strip()
        except Exception as e:
            print(f"Retrieval error for question: {question}\nError: {e}")
            continue

        # Generate answers
        def get_answer(context):
            if not context:
                return ""
            try:
                result = qa_pipeline(question=question, context=context)
                return result["answer"] if isinstance(result, dict) else ""
            except Exception as e:
                print(f"QA pipeline error: {e}")
                return ""

        answers = {
            "literal": get_answer(literal_context),
            "pragmatic": get_answer(pragmatic_context),
            "retrieved": get_answer(retrieved_context)
        }

        # Compute scores
        gold_ex = dspy.Example(question=question, response=gold, 
                             inputs={"context": retrieved_context})
        
        scores = {}
        for approach, answer in answers.items():
            pred_ex = dspy.Example(question=question, response=answer,
                                 inputs={"context": retrieved_context})
            try:
                scores[approach] = metric(gold_ex, pred_ex)
            except Exception as e:
                print(f"Scoring error for {approach}: {e}")
                scores[approach] = 0.0

        results.append({
            "topic": doc.get("topic", "unknown"),
            "question": question,
            "gold": gold,
            "ans_literal": answers.get("literal", ""),
            "ans_pragmatic": answers.get("pragmatic", ""),
            "ans_retrieved": answers.get("retrieved", ""),
            "f1_literal": scores.get("literal", 0.0),
            "f1_pragmatic": scores.get("pragmatic", 0.0),
            "f1_retrieved": scores.get("retrieved", 0.0)
        })
        
    return pd.DataFrame(results)

# Setup components
print("Setting up evaluation pipeline...")
qa_pipeline = setup_qa_pipeline(device='cuda')
metric = SemanticF1(decompositional=True)
val_data = load_validation_data()
num_samples = 100
if not val_data:
    print("No validation data loaded!")
else:
    print(f"Loaded {len(val_data)} validation documents")
    
    # Run evaluation
    print("\nEvaluating first questions from validation set...")
    results_df = evaluate_qa_approaches(val_data, search, qa_pipeline, metric,max_samples=num_samples)
    
    # Show aggregate results
    print("\nAggregate Results:")
    agg_scores = results_df[["f1_literal", "f1_pragmatic", "f1_retrieved"]].mean()
    print("\nMean F1 across all questions:")
    print(agg_scores)
    
    # Show sample predictions
    print("\nSample Predictions (first 3):")
    for _, row in results_df.head(3).iterrows():
        print("\nQuestion:", row["question"])
        print("Gold:", row["gold"][:200], "..." if len(row["gold"]) > 200 else "")
        print(f"Literal (F1 {row['f1_literal']:.2f}):", row["ans_literal"])
        print(f"Pragmatic (F1 {row['f1_pragmatic']:.2f}):", row["ans_pragmatic"])
        print(f"Retrieved (F1 {row['f1_retrieved']:.2f}):", row["ans_retrieved"])
        print("-" * 80)

# Save results
results_df.to_csv("traditional_qa_results.csv", index=False)
print("\nFull results saved to traditional_qa_results.csv")

Setting up evaluation pipeline...


Device set to use cuda:0


Loaded 179 validation documents

Evaluating first questions from validation set...


Evaluating QA approaches:   0%|          | 0/100 [00:00<?, ?it/s]

Directory not found: /home/lotemsakira/PycharmProjects/PragmatiCQA-sources/A Nightmare on Elm Street (2010 film)
No corpus found for topic 'A Nightmare on Elm Street (2010 film)', skipping question: who is freddy krueger?
Directory not found: /home/lotemsakira/PycharmProjects/PragmatiCQA-sources/A Nightmare on Elm Street (2010 film)
No corpus found for topic 'A Nightmare on Elm Street (2010 film)', skipping question: who was the star on this movie?
Directory not found: /home/lotemsakira/PycharmProjects/PragmatiCQA-sources/A Nightmare on Elm Street (2010 film)
No corpus found for topic 'A Nightmare on Elm Street (2010 film)', skipping question: What is the movie about?
Directory not found: /home/lotemsakira/PycharmProjects/PragmatiCQA-sources/A Nightmare on Elm Street (2010 film)
No corpus found for topic 'A Nightmare on Elm Street (2010 film)', skipping question: Who directed the new film?


...

Scoring error for literal: Both structured output format and JSON mode failed. Please choose a model that supports `response_format` argument. Original error: litellm.AuthenticationError: AuthenticationError: XaiException - {"error":{"code":401,"message":"No or an invalid authentication header was provided on the request. Please make sure you pass your API key in the corresponding HTTP header."}}


2025/09/06 10:06:10 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


Scoring error for pragmatic: Both structured output format and JSON mode failed. Please choose a model that supports `response_format` argument. Original error: litellm.AuthenticationError: AuthenticationError: XaiException - {"error":{"code":401,"message":"No or an invalid authentication header was provided on the request. Please make sure you pass your API key in the corresponding HTTP header."}}


2025/09/06 10:06:18 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


Scoring error for retrieved: Both structured output format and JSON mode failed. Please choose a model that supports `response_format` argument. Original error: litellm.AuthenticationError: AuthenticationError: XaiException - {"error":{"code":401,"message":"No or an invalid authentication header was provided on the request. Please make sure you pass your API key in the corresponding HTTP header."}}

Aggregate Results:

Mean F1 across all questions:
f1_literal      0.382198
f1_pragmatic    0.344409
f1_retrieved    0.020614
dtype: float64

Sample Predictions (first 3):

Question: Is the Batman comic similar to the movies?
Gold: I would say the movie and comics has same story line, as Batmans parents were the most wealthy folks in Gotham city, and they were killd while returning from a function by a small time criminal called ...
Literal (F1 0.40): Gotham City socialites
Pragmatic (F1 0.40): his parents were killed by a small-time criminal named Joe Chill
Retrieved (F1 0.00): The most con

Part 4.4 — LLM Multi-Step Prompting Approach


In [9]:
import itertools
from dspy.evaluate import SemanticF1

class CooperativeRAG(dspy.Module):
    def __init__(self, retriever_k=5):
        super().__init__()
        self.retriever_k = retriever_k
        self.summarize = dspy.ChainOfThought('history -> summary')
        self.infer_need = dspy.ChainOfThought('summary, question -> need')
        self.gen_coop_q = dspy.Predict('summary, question, need -> coop_question')
        self.respond = dspy.ChainOfThought('context, question, summary, need -> response')

    def forward(self, question, history, topic):
        # Prepare topic-specific retriever
        topic_corpus = read_html_files(topic)
        if not topic_corpus:
            # Fallback to empty context
            hist_str = "\n".join([f"Q: {h.get('q','')}\nA: {h.get('a','')}" for h in history]) if history else ""
            summary_ex = self.summarize(history=hist_str)
            summary = getattr(summary_ex, 'summary', getattr(summary_ex, 'response', str(summary_ex)))
            need_ex = self.infer_need(summary=summary, question=question)
            need = getattr(need_ex, 'need', getattr(need_ex, 'response', str(need_ex)))
            return self.respond(context="", question=question, summary=summary, need=need)

        topic_search = dspy.retrievers.Embeddings(embedder=embedder, corpus=topic_corpus, k=self.retriever_k)

        # Summarize history and infer pragmatic need
        hist_str = "\n".join([f"Q: {h.get('q','')}\nA: {h.get('a','')}" for h in history]) if history else ""
        summary_ex = self.summarize(history=hist_str)
        summary = getattr(summary_ex, 'summary', getattr(summary_ex, 'response', str(summary_ex)))
        need_ex = self.infer_need(summary=summary, question=question)
        need = getattr(need_ex, 'need', getattr(need_ex, 'response', str(need_ex)))

        # Generate a cooperative follow-up question and retrieve again
        coop_q_ex = self.gen_coop_q(summary=summary, question=question, need=need)
        coop_q = getattr(coop_q_ex, 'coop_question', getattr(coop_q_ex, 'text', str(coop_q_ex)))

        ctx_main = topic_search(question).passages
        ctx_coop = topic_search(coop_q).passages if coop_q else []
        # merge contexts preserving order and uniqueness
        merged_ctx = list(dict.fromkeys(list(itertools.chain(ctx_main, ctx_coop))))
        merged_text = " ".join(merged_ctx)

        return self.respond(context=merged_text, question=question, summary=summary, need=need)


In [10]:

def evaluate_llm_first_questions(val_data, metric, retriever_k=5, max_samples=None):
    rows = []
    samples = val_data[:max_samples] if max_samples else val_data
    for doc in tqdm(samples, desc="LLM First-Question Eval"):
        if not doc.get('qas'):
            continue
        qa = doc['qas'][0]
        topic = doc.get('topic', '')
        question = qa['q']
        gold = qa['a']
        try:
            model = CooperativeRAG(retriever_k=retriever_k)
            pred = model(question=question, history=[], topic=topic)
            answer = getattr(pred, 'response', getattr(pred, 'text', str(pred)))
        except Exception as e:
            print(f"Error generating LLM answer: {e}")
            answer = ""
        gold_ex = dspy.Example(question=question, response=gold)
        pred_ex = dspy.Example(question=question, response=answer)
        try:
            f1 = metric(gold_ex, pred_ex)
        except Exception as e:
            print(f"SemanticF1 error: {e}")
            f1 = 0.0
        rows.append({
            'topic': topic,
            'question': question,
            'gold': gold,
            'answer': answer,
            'f1': f1
        })
    return pd.DataFrame(rows)


In [11]:
def evaluate_llm_full_conversations(val_data, metric, retriever_k=5, max_conversations=None, max_turns=None):
    rows = []
    convs = val_data[:max_conversations] if max_conversations else val_data
    for doc in tqdm(convs, desc="LLM Full-Conversation Eval"):
        qas = doc.get('qas', [])
        if not qas:
            continue
        topic = doc.get('topic', '')
        history = []
        model = CooperativeRAG(retriever_k=retriever_k)
        for turn_idx, qa in enumerate(qas):
            if max_turns is not None and turn_idx >= max_turns:
                break
            question = qa['q']
            gold = qa['a']
            try:
                pred = model(question=question, history=history, topic=topic)
                answer = getattr(pred, 'response', getattr(pred, 'text', str(pred)))
            except Exception as e:
                print(f"Error generating LLM answer: {e}")
                answer = ""
            gold_ex = dspy.Example(question=question, response=gold)
            pred_ex = dspy.Example(question=question, response=answer)
            try:
                f1 = metric(gold_ex, pred_ex)
            except Exception as e:
                print(f"SemanticF1 error: {e}")
                f1 = 0.0
            rows.append({
                'topic': topic,
                'turn': turn_idx,
                'question': question,
                'gold': gold,
                'answer': answer,
                'f1': f1
            })
            history.append({'q': question, 'a': answer})
    return pd.DataFrame(rows)


In [12]:
print("\nRunning Part 4.4 evaluations...")
metric_llm = SemanticF1(decompositional=True)

if 'val_data' not in globals():
    val_data = load_validation_data()

if not val_data:
    print("No validation data loaded for LLM evaluation!")
else:
    print(f"Loaded {len(val_data)} validation documents")
    llm_first_df = evaluate_llm_first_questions(val_data, metric_llm, retriever_k=topk_docs_to_retrieve, max_samples=50)
    print("First-question LLM mean F1:", llm_first_df['f1'].mean())
    llm_first_df.to_csv('llm_first_results.csv', index=False)

    llm_full_df = evaluate_llm_full_conversations(val_data, metric_llm, retriever_k=topk_docs_to_retrieve, max_conversations=20, max_turns=5)
    print("Full-conversation (subset) LLM mean F1:", llm_full_df['f1'].mean())
    llm_full_df.to_csv('llm_full_results.csv', index=False)

print("Part 4.4 evaluations completed.")



Running Part 4.4 evaluations...
Loaded 179 validation documents


LLM First-Question Eval:   0%|          | 0/50 [00:00<?, ?it/s]

Directory not found: /home/lotemsakira/PycharmProjects/PragmatiCQA-sources/A Nightmare on Elm Street (2010 film)


2025/09/06 10:06:26 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


Error generating LLM answer: Both structured output format and JSON mode failed. Please choose a model that supports `response_format` argument. Original error: litellm.AuthenticationError: AuthenticationError: XaiException - {"error":{"code":401,"message":"No or an invalid authentication header was provided on the request. Please make sure you pass your API key in the corresponding HTTP header."}}


...

SemanticF1 error: Both structured output format and JSON mode failed. Please choose a model that supports `response_format` argument. Original error: litellm.AuthenticationError: AuthenticationError: XaiException - {"error":{"code":401,"message":"No or an invalid authentication header was provided on the request. Please make sure you pass your API key in the corresponding HTTP header."}}


2025/09/06 11:13:28 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


Error generating LLM answer: Both structured output format and JSON mode failed. Please choose a model that supports `response_format` argument. Original error: litellm.AuthenticationError: AuthenticationError: XaiException - {"error":{"code":401,"message":"No or an invalid authentication header was provided on the request. Please make sure you pass your API key in the corresponding HTTP header."}}


2025/09/06 11:13:36 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


SemanticF1 error: Both structured output format and JSON mode failed. Please choose a model that supports `response_format` argument. Original error: litellm.AuthenticationError: AuthenticationError: XaiException - {"error":{"code":401,"message":"No or an invalid authentication header was provided on the request. Please make sure you pass your API key in the corresponding HTTP header."}}
Full-conversation (subset) LLM mean F1: 0.0
Part 4.4 evaluations completed.
